In [ ]:
# (Minimal) Perfect Hash Functions Generator (key, value) value in this code is the key counter during reading but can be any number

# implementing the MOS Algorithm II CACM92 , and Amjad M Daoud Thesis 1993 at VT; 
# based on Steve Hanof implementation http://stevehanov.ca/blog/index.php?id=119.

# Download as http://iswsa.acm.org/mphf/mphf.py

# You need Python; runs linearly even on a Android phone; it runs without modifications at http://www.compileonline.com/execute_python_online.php

# For minimal perfect hashing use:  size = len(dict)

In [90]:
!pip install pyhash

     |████████████████████████████████| 604kB 7.7MB/s 
  Created wheel for pyhash: filename=pyhash-0.9.3-cp36-cp36m-linux_x86_64.whl size=2206689 sha256=9899d19532d1df77040fe7c1a4d1f5173222d754dc8cb82c141db080e5e9bb9e
  Stored in directory: /root/.cache/pip/wheels/0b/6d/31/936fd0c841c52f7afcb77290e2de18fd04d88325853787b36e
Successfully built pyhash


In [92]:
import sys
import math
import pyhash

In [93]:
hasher = pyhash.fnv1_32()

In [134]:
# first level simple hash ... used to disperse patterns using random d values

def hash( d, str ):

    #if d == 0: d = 0x01000193

    if d == 0: d =   0x811C9DC5

    # Use the FNV-1a hash
    #d = hasher(str)

    for c in str:

        #h = (h ^ p[i]) * 16777619
        d = ( (d * 0x01000193) ^ ord(c) ) & 0xffffffff;

        #d = d ^ ord(c) * 16777619 & 0xffffffff

    return d 

In [112]:
def isprime(x):

	x = abs(int(x))

	if x < 2:

		return "Less 2", False

	elif x == 2:

		return True

	elif x % 2 == 0:

		return False	

	else:

		for n in range(3, int(x**0.5)+2, 2):

			if x % n == 0:

				return False

		return True

In [113]:
def nextprime(x):

    while ( True ):

       if isprime(x): break

       x += 1

    return x  

In [114]:
def xrange(x):

    return iter(range(x))

In [149]:
# create PHF with MOS(Map,Order,Search), g is specifications array

def CreatePHF( dict ):

    size = len(dict) 

    size = nextprime(len(dict))

    print("Size = %d" % (size))

    #nextprime(int(size/(6*math.log(size,2)))) 
    #c=4 corresponds to 4 bits/key
    # for fast construction use size/5
    # for faster construction use gsize=size
    gsize = int(size) // 10

    print("G array size = %d" % (gsize))

    sys.stdout.flush()

    #Step 1: Mapping

    patterns = [ [] for i in range(gsize) ]

    g = [0] * gsize #initialize g

    values = [None] * int(size) #initialize values

    for key in dict:

        patterns[hash(0, key) % gsize].append( key )

    print(patterns)

    # Step 2: Sort patterns in descending order and process  
    patterns.sort( key= len, reverse=True )        

    print(patterns)

    for b in xrange( gsize ):

        pattern = patterns[b]

        if len(pattern) <= 1: break
       
        d = 1

        item = 0

        slots = []

    # Step 3: rotate patterns and search for suitable displacement

        while item < len(pattern):

            slot = int(hash( d, pattern[item] ) % size)

            if values[slot] != None or slot in slots:

                d += 1

                if d < 0 : break

                item = 0

                slots = []

            else:

                slots.append( slot )

                item += 1

        if d < 0: 

           print("failed")

           return

        g[hash(0, pattern[0]) % gsize] = d

        for i in range(len(pattern)):

            values[slots[i]] = pattern[i]

        if ( b % 10 ) == 0:

          print("%d: pattern %d processed" % (b,len(pattern)))

          sys.stdout.flush()

    # Process patterns with one key and use a negative value of d 

    freelist = []

    for i in xrange(size): 

        if values[i] == None: freelist.append( i )

    for b in xrange(gsize):

        pattern = patterns[b]

        if len(pattern) == 0: break

        if len(pattern) > 1: continue;

        slot = freelist.pop()

        # subtract one to handle slot zero

        g[hash(0, pattern[0]) % gsize] = -slot-1 

        values[slot] = pattern[0]

        if (b % 10) == 0:

          print("%d: pattern %d processed" % (b,len(pattern)))

          sys.stdout.flush()

    print("PHF succeeded")

    return (g, values)        

In [116]:
# Look up a value in the hash table, defined by g and V.

def lookup( g, V, key ):

    d = g[hash(0,key) % len(g)]

    if d < 0: return V[-d-1]

    return V[hash(d, key) % len(V)]

In [150]:
# main program

#reading keyset size is given by num

#DICTIONARY = "/etc/dictionaries-common/words"

#num = 12

#print("Reading %d Unix user dict words"% (num))

stopwords = ['a', 'about', 'above', 'across', 'after', 'afterwards']
stopwords += ['again', 'against', 'all', 'almost', 'alone', 'along']
stopwords += ['already', 'also', 'although', 'always', 'am', 'among']
stopwords += ['amongst', 'amoungst', 'amount', 'an', 'and', 'another']
stopwords += ['any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere']
stopwords += ['are', 'around', 'as', 'at', 'back', 'be', 'became']
stopwords += ['because', 'become', 'becomes', 'becoming', 'been']
stopwords += ['before', 'beforehand', 'behind', 'being', 'below']
stopwords += ['beside', 'besides', 'between', 'beyond', 'bill', 'both']
stopwords += ['bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant']
stopwords += ['co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de']
stopwords += ['describe', 'detail', 'did', 'do', 'done', 'down', 'due']
stopwords += ['during', 'each', 'eg', 'eight', 'either', 'eleven', 'else']
stopwords += ['elsewhere', 'empty', 'enough', 'etc', 'even', 'ever']
stopwords += ['every', 'everyone', 'everything', 'everywhere', 'except']
stopwords += ['few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first']
stopwords += ['five', 'for', 'former', 'formerly', 'forty', 'found']
stopwords += ['four', 'from', 'front', 'full', 'further', 'get', 'give']
stopwords += ['go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her']
stopwords += ['here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers']
stopwords += ['herself', 'him', 'himself', 'his', 'how', 'however']
stopwords += ['hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed']
stopwords += ['interest', 'into', 'is', 'it', 'its', 'itself', 'keep']
stopwords += ['last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made']
stopwords += ['many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine']
stopwords += ['more', 'moreover', 'most', 'mostly', 'move', 'much']
stopwords += ['must', 'my', 'myself', 'name', 'namely', 'neither', 'never']
stopwords += ['nevertheless', 'next', 'nine', 'no', 'nobody', 'none']
stopwords += ['noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of']
stopwords += ['off', 'often', 'on','once', 'one', 'only', 'onto', 'or']
stopwords += ['other', 'others', 'otherwise', 'our', 'ours', 'ourselves']
stopwords += ['out', 'over', 'own', 'part', 'per', 'perhaps', 'please']
stopwords += ['put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed']
stopwords += ['seeming', 'seems', 'serious', 'several', 'she', 'should']
stopwords += ['show', 'side', 'since', 'sincere', 'six', 'sixty', 'so']
stopwords += ['some', 'somehow', 'someone', 'something', 'sometime']
stopwords += ['sometimes', 'somewhere', 'still', 'such', 'system', 'take']
stopwords += ['ten', 'than', 'that', 'the', 'their', 'them', 'themselves']
stopwords += ['then', 'thence', 'there', 'thereafter', 'thereby']
stopwords += ['therefore', 'therein', 'thereupon', 'these', 'they']
stopwords += ['thick', 'thin', 'third', 'this', 'those', 'though']
stopwords += ['three', 'through', 'throughout', 'thru', 'thus', 'to']
stopwords += ['together', 'too', 'top', 'toward', 'towards', 'twelve']
stopwords += ['twenty', 'two', 'un', 'under', 'until', 'up', 'upon']
stopwords += ['us', 'very', 'via', 'was', 'we', 'well', 'were', 'what']
stopwords += ['whatever', 'when', 'whence', 'whenever', 'where']
stopwords += ['whereafter', 'whereas', 'whereby', 'wherein', 'whereupon']
stopwords += ['wherever', 'whether', 'which', 'while', 'whither', 'who']
stopwords += ['whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with']
stopwords += ['within', 'without', 'would', 'yet', 'you', 'your']
stopwords += ['yours', 'yourself', 'yourselves']

#line = 1

#for key in open(DICTIONARY, "rt").readlines():

#    dict[key.strip()] = line

#    line += 1

#    if line > num: break

#creating phf

print("Creating perfect hash for the first %d stopwords"% (len(stopwords)))

(g, V) = CreatePHF( stopwords )

#printing phf specification

print("Printing g[] for dict")

print(g)

#fast verification for few (key,value) count given by num1

num1 = 5

print("Verifying hash values for the first %d words"% (num1))

l = 1

for key in stopwords:

    line = lookup( g, V, key.strip() )

    print("Word %s occurs on line %s" % (key.strip(), line))

    l += 1

    if l > num1: break

Creating perfect hash for the first 321 stopwords
Size = 331
G array size = 33
[['another', 'five', 'for', 'her', 'made', 'must', 'seem', 'take', 'throughout', 'toward', 'yourself'], ['anywhere', 'cry', 'elsewhere', 'fifteen', 'hereby', 'meanwhile', 'seems', 'thereafter', 'those', 'together', 'were', 'whom', 'you'], ['anyway', 'because', 'etc', 'even', 'least', 'noone', 'only', 'ours', 'seeming', 'somehow', 'somewhere', 'under'], ['be', 'front', 'here', 'while'], ['always', 'among', 'beforehand', 'each', 'enough', 'every', 'him', 'move', 'others'], ['against', 'beside', 'did', 'full', 'herein', 'other', 'rather', 'thus', 'to', 'wherein'], ['already', 'amoungst', 'do', 'give', 'inc', 'next', 'thereupon', 'third', 'via', 'whereby', 'who'], ['after', 'become', 'had', 'indeed', 'moreover', 'no', 'thereby'], ['bottom', 'due', 'fifty', 'hasnt', 'hereafter', 'might', 'never', 'perhaps', 'was'], ['around', 'both', 'forty', 'further', 'get', 'hundred', 'six', 'something', 'top', 'what', 'whole'